# 3.0 Model Fine-tuning

In [ ]:
# Downloading the preprocessed training data
!gdown 1I_PLI0-3GByjpg7_HyZVzd3FhDj-hGVq

Downloading...
From: https://drive.google.com/uc?id=1I_PLI0-3GByjpg7_HyZVzd3FhDj-hGVq
To: /content/combined.txt
100% 9.06M/9.06M [00:00<00:00, 87.3MB/s]


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.8 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
# Doing all the necessary imports
import os
from transformers import (
    AutoModelWithLMHead,
    AutoConfig,
    Trainer,
    AutoTokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    TrainingArguments
)
from transformers.trainer import TrainingArguments
from transformers.data.data_collator import DataCollatorForLanguageModeling
from transformers import pipeline, AutoModelWithLMHead, AutoTokenizer

In [ ]:
train_file = "/content/combined.txt"

In [ ]:
def modelTrainer(train_file, epochs, model='gpt2', batch_size=8, cache_dir='cache'):
    model = AutoModelWithLMHead.from_pretrained(model)
    tokenizer = AutoTokenizer.from_pretrained('gpt2')
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm = False)
    train_dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = train_file,
        block_size = 256
    )
    training_args = TrainingArguments(
        output_dir = "gpt2_fine_tune",
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        warmup_steps=100,
        save_steps = 500,
        logging_steps = 200
    )
    trainer = Trainer(
        model = model,
        args = training_args,
        data_collator = data_collator,
        train_dataset = train_dataset,
    )

    trainer.train()
    trainer.save_model()

In [ ]:
epochs = 3

modelTrainer(train_file, epochs)

Token indices sequence length is longer than the specified maximum sequence length for this model (2864525 > 1024). Running this sequence through the model will result in indexing errors


Step,Training Loss
200,2.241100
400,2.036500
600,1.992600
800,1.966500
1000,1.959900
1200,1.947100
1400,1.931900
1600,1.838100
1800,1.847100
2000,1.843400


In [ ]:
model = AutoModelWithLMHead.from_pretrained("/content/gpt2_fine_tune")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
pad_token_id = generator.tokenizer.eos_token_id

In [ ]:
def detoxify(input_sentence):
    p = generator('<s>' + input_sentence + '</s>>>>><p>', pad_token_id=generator.tokenizer.eos_token_id)
    return p[0]['generated_text'].split('</s>>>>><p>')[1].split('</p>')[0]

In [ ]:
test = ['His father would have used a booming voice to wrench a reply from this stupid machine.',
       'You have to send those idiots back in.',
       'Salina could be with that stupid cop.',
       "And don't let those idiots in radiology hold you up.",
       'My idiot friend here brought marijuana... - on the plane.',
       'That stupid couple told her to break up with you.',
       'The idiot, Max. He never should have sold it to you guys.',
       "Fuck you, soft. You're panicking",
       "It's coz of those two idiots. They escaped through a tunnel.",
       'You have to let this stupid plane crash make the decision for you.',
       "Audrey Cruz is an idiot. He's the boss.",
       "Why don't you get a job. Instead of playing golf with your stupid buddies?",
       'How do you like your stupid game now, Tom?',
       'Think about that shit, dawg.',
       'You idiots! You have betrayed the revolution.',
       'Why is this idiot Silencer shouting so much?',
       'Just like the rest of the stupid Irish in this country.',
       'Your shit is so tired, Justice.',
       'More than the stupid senators and congressmen that passed it.',
       'How do you possibly know, you stupid woman?']

In [ ]:
answers = list()
for t in test:
    answers.append(detoxify(t))

In [ ]:
answers

['his father would have given him a ringing voice, and he would have spoken.',
 'now you have to send them back.',
 'sala can be with the cop.',
 "and don't let them take care of it in medicine.",
 'I have a friend who brought me a weed in the plane.',
 'they told her to break up with you.',
 "the poor guy, he didn't want to sell it to you.",
 "you're panicking.",
 "it's Cozy, and they escaped through the tunnel.",
 'you have to make the decision for the plane.',
 'Audrey Cruz is the boss.',
 "why don't you show some respect to your friends and stop golf?",
 'how do you like playing your game, Tom?',
 'think about it, man.',
 'you betrayed the revolution.',
 'why is it so loud?',
 "just like we're completely in Ireland.",
 'the weather is so bad, Justice.',
 'more than the lawmakers and congressmen who approved it.',
 'how do you know you could know that?']

## Generating test output

In [ ]:
!gdown 1LPBDo0rDRl4wkiFvN0wVbhys7BdHW1SR

Downloading...
From: https://drive.google.com/uc?id=1LPBDo0rDRl4wkiFvN0wVbhys7BdHW1SR
To: /content/test.csv
100% 1.95M/1.95M [00:00<00:00, 188MB/s]


In [ ]:
import pandas as pd
df_test = pd.read_csv('/content/test.csv')

In [ ]:
df_test.head()

,reference,translation
0,Now I say to you the last words my father said...,now I'll tell you the last words my father sai...
1,And that's on two cans of fucking cider.,thanks to those two cans of beer.
2,"“Not that it’s any of your damn business, but ...","""not that it's any of your business, but my wi..."
3,I want more life... ...Fucker!,I want a long life.
4,The whole goddamn Army is falling apart.,the whole army is breaking up.


In [ ]:
len(df_test)

20000

In [ ]:
from tqdm import tqdm

predictions = list()
answers = []
for t in tqdm(df_test["reference"][:100]):
    answers.append(detoxify(t))

In [ ]:
from tqdm import tqdm

xx = 0
for i in tqdm(range(10000000)):
    xx += i

100%|██████████| 10000000/10000000 [00:07<00:00, 1329065.00it/s]


In [ ]:
with open(r'test_answers.txt', 'w') as fp:
    for line in answers:
        # write each item on a new line
        fp.write("%s\n" % line)
    print('Done')

Done


In [ ]:
len(df_test["reference"][:100])

100

In [ ]:
len()

In [ ]:
!mkdir model
!cp gpt2_fine_tune/config.json model/config.json
!cp gpt2_fine_tune/generation_config.json model/generation_config.json
!cp gpt2_fine_tune/pytorch_model.bin model/pytorch_model.bin
!cp gpt2_fine_tune/training_args.bin model/training_args.bin

mkdir: cannot create directory ‘model’: File exists
  adding: model/ (stored 0%)


In [ ]:
import shutil

shutil.make_archive('model', 'zip', '', 'model')

'/content/model.zip'